## 2. Source Data Merging and Folding

Last Updated: December 11, 2018

In [114]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import SVG
from sklearn import preprocessing
import os
from os.path import join, dirname
from os.path import dirname, join
from tqdm import tqdm_notebook
import pandas as pd
from pandas import pivot
from pandas import tools
import numpy as np
import cv2
import re
import io
import sys 
import pip
import csv
from sklearn.model_selection import train_test_split
import glob
import random
import warnings
from pandas import DataFrame   
import pickle
import cloudpickle
import azureml
import string
from scipy import stats
import pyparsing
import tempfile  
import logging
import graphviz

In [115]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Working Directory Definition

In [116]:
os.chdir('D://MAFData/v2data')
base_dir = 'D://MAFData/v2data'
WorkingDir = os.path.join(base_dir, 'Workingdir')

print(base_dir)
os.getcwd()

D://MAFData/v2data


'D:\\MAFData\\v2data'

### Read in Cleaned Source Data

In [117]:
import pprint, pickle
PATH = 'D://MAFData//'
os.chdir('D://MAFData')
os.getcwd()

import datetime
t1 = datetime.datetime.now(); print(t1)

with open(PATH + 'Portfolio_cleaned.pkl', 'rb') as f:
    Portfolio_cleaned = pickle.load(f)

t2 = datetime.datetime.now(); print(t2) 
time_spent = t2-t1; print(time_spent)

'D:\\MAFData'

2018-12-11 19:05:37.278413
2018-12-11 19:06:08.196957
0:00:30.918544


In [118]:
Portfolio_cleaned.shape[0]/365
Portfolio_cleaned.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False], inplace=True)
Portfolio_cleaned.reset_index(drop=True, inplace=True)

81187.0

### Count of 'ATTRITION_FLAG' == 'VOLUNTARY'

    extract Account_Number of VOLUNTARY



In [119]:
Portfolio_cleaned = Portfolio_cleaned.copy()
Portfolio_cleaned.shape[0]/365


81187.0

In [120]:
df_VOLUNTARY_Account_Number = pd.DataFrame(Portfolio_cleaned[Portfolio_cleaned['ATTRITION_FLAG'] == 'VOLUNTARY'].Account_Number)
df_VOLUNTARY_Account_Number.shape
df_VOLUNTARY_Account_Number = pd.DataFrame(df_VOLUNTARY_Account_Number)


(6906, 1)

In [121]:
len(df_VOLUNTARY_Account_Number)


6906

#### Remove duplicated accounts from df

In [122]:
VOLUNTARY_Account_Number_dup = df_VOLUNTARY_Account_Number[df_VOLUNTARY_Account_Number.duplicated(keep=False)]
VOLUNTARY_Account_Number_dup.shape


(36, 1)

In [123]:
Portfolio_cleaned.shape
Portfolio_cleaned.shape[0]/365
Portfolio_cleaned = Portfolio_cleaned[~Portfolio_cleaned['Account_Number'].isin(list(VOLUNTARY_Account_Number_dup['Account_Number']))]
Portfolio_cleaned.shape[0]/365

(29633255, 24)

81187.0

81169.0

In [124]:
unique_accounts_count = len(Portfolio_cleaned['Account_Number'].unique())
unique_accounts_count
df = Portfolio_cleaned


81169

In [125]:
len(df_VOLUNTARY_Account_Number['Account_Number'].unique())
unique_accounts_count
len(df_VOLUNTARY_Account_Number['Account_Number'].unique())/unique_accounts_count

6888

81169

0.08485998349123434

### Create List of Open Account Numbers to Resample Timespan

In [126]:
df_OPEN = df.loc[~df['Account_Number'].isin(df_VOLUNTARY_Account_Number['Account_Number'].unique())]

In [127]:
df_OPEN.isnull().sum()

Account_Number                         0
MISDATE                                0
ATTRITION_FLAG                         0
ACCOUNT_CREATION_DATE                  0
ACTUAL_CURRENT_BALANCE                 0
Credit_Limit                           0
LAST_ACTIVITY_DATE                747368
Last_Statement_Closing_Balance         0
Last_Statement_Date                    0
Last_Statement_Minimum_Due             0
Monthly_Salary                         0
Months_On_Book                         0
Next_Statement_Date                    0
Monthly_Salary                         0
Nationality_Group                      0
Next_Statement_Date                    0
No_Of_Due_Stmt                         0
Product                                0
APR_Rate                               0
Product_Fee_Type                       0
Product_Category                       0
Days_On_Book                           0
Days_From_Statement                    0
Days_from_Last_Activity                0
dtype: int64

In [128]:
unique_OPEN = df_OPEN['Account_Number'].unique()
len(unique_OPEN)
unique_Account_Number = df['Account_Number'].unique()
len(unique_Account_Number)
unique_VOLUNTARY = df_VOLUNTARY_Account_Number['Account_Number'].unique()
len(unique_VOLUNTARY)

#assert len(unique_Account_Number) == (len(unique_OPEN) + len(unique_VOLUNTARY))

74299

81169

6888

In [129]:
type(df_OPEN)
df.shape
df_OPEN.shape
df_OPEN.shape[0]/365


pandas.core.frame.DataFrame

(29626685, 24)

(27119135, 24)

74299.0

### Get range of MISDATE where df_OPEN

extract earliest possible start dates for each account

In [130]:
# Check min date
min_dates = df_OPEN.groupby('Account_Number')['MISDATE'].min()
# Check max date
max_dates = df_OPEN.groupby('Account_Number')['MISDATE'].max()

In [131]:
to_dates = max_dates
from_dates = min_dates + pd.DateOffset(days=130)

In [132]:
dates = pd.DataFrame(to_dates); dates.shape; dates.columns; 
dates.reset_index(inplace=True); #dates
dates.rename(columns={"MISDATE": "to_dates"}, inplace=True)

(74299, 1)

Index(['MISDATE'], dtype='object')

In [133]:
dates['from_dates'] = pd.DataFrame(from_dates.values)
dates.shape
dates = dates[dates['to_dates'] == '2017-09-30']
dates.shape
df_OPEN.shape[0]/365
len(df_OPEN['Account_Number'].unique())

(74299, 3)

(74239, 3)

74299.0

74299

In [134]:
dates.isnull().sum()

Account_Number    0
to_dates          0
from_dates        0
dtype: int64

### Create a Random Selection of `start_date` for Sample

In [135]:
dates.columns
dates.shape
dates.to_dates.value_counts()

Index(['Account_Number', 'to_dates', 'from_dates'], dtype='object')

(74239, 3)

2017-09-30    74239
Name: to_dates, dtype: int64

In [136]:
from random import randint
import datetime
datetime.timedelta(randint(1,234))
len(dates['to_dates'])
from random import randint
import datetime
dates['randomint']=""
dates['randomint'] = np.random.choice(range(1,234),len(dates),replace=True)


datetime.timedelta(137)

74239

In [137]:
dates['random_end_dt']=dates['to_dates']-pd.to_timedelta(dates['randomint'],unit='d')
dates.head(2)
dates.shape

,Account_Number,to_dates,from_dates,randomint,random_end_dt
0,MAFF_ACCT_0000002125,2017-09-30,2017-02-08,80,2017-07-12
1,MAFF_ACCT_0000002171,2017-09-30,2017-02-08,118,2017-06-04


(74239, 5)

In [138]:
df_OPEN1 = pd.merge(df_OPEN, dates, how='left',on='Account_Number')

del df_OPEN1['to_dates']
del df_OPEN1['from_dates']
df_OPEN1.shape
df_OPEN1.head(3)

(27119135, 26)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category,Days_On_Book,Days_From_Statement,Days_from_Last_Activity,randomint,random_end_dt
0,MAFF_ACCT_0000002125,2017-09-30,OPEN,2012-10-24,12554.14,18000,2017-09-30,17391,2017-09-10,869,...,0,007,LOW,NON FEE,CASHBACK,1802,20,0,80.0,2017-07-12
1,MAFF_ACCT_0000002125,2017-09-29,OPEN,2012-10-24,12454.14,18000,2017-09-29,17391,2017-09-10,869,...,0,007,LOW,NON FEE,CASHBACK,1801,19,0,80.0,2017-07-12
2,MAFF_ACCT_0000002125,2017-09-28,OPEN,2012-10-24,12291.14,18000,2017-09-28,17391,2017-09-10,869,...,0,007,LOW,NON FEE,CASHBACK,1800,18,0,80.0,2017-07-12


In [139]:
df_OPEN2 = df_OPEN1[df_OPEN1['MISDATE'] <= df_OPEN1['random_end_dt']]
df_OPEN2.head(2)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category,Days_On_Book,Days_From_Statement,Days_from_Last_Activity,randomint,random_end_dt
80,MAFF_ACCT_0000002125,2017-07-12,OPEN,2012-10-24,6911.48,18000,2017-07-12,6775,2017-07-10,338,...,0,007,LOW,NON FEE,CASHBACK,1722,2,0,80.0,2017-07-12
81,MAFF_ACCT_0000002125,2017-07-11,OPEN,2012-10-24,6822.48,18000,2017-07-11,6775,2017-07-10,338,...,1,007,LOW,NON FEE,CASHBACK,1721,1,0,80.0,2017-07-12


In [140]:
del df_OPEN2['randomint']

In [141]:
df_OPEN2 = df_OPEN2.reset_index(drop=True)
df_OPEN2 = df_OPEN2.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False])
df_OPEN2 = df_OPEN2.reset_index(drop=True)
df_OPEN2 = df_OPEN2.groupby('Account_Number').head(130)
df_OPEN2 = df_OPEN2.reset_index(drop=True)
df_OPEN2.shape
df_OPEN2.head(2)

(9651070, 25)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category,Days_On_Book,Days_From_Statement,Days_from_Last_Activity,random_end_dt
0,MAFF_ACCT_0000002125,2017-07-12,OPEN,2012-10-24,6911.48,18000,2017-07-12,6775,2017-07-10,338,...,2017-08-10,0,007,LOW,NON FEE,CASHBACK,1722,2,0,2017-07-12
1,MAFF_ACCT_0000002125,2017-07-11,OPEN,2012-10-24,6822.48,18000,2017-07-11,6775,2017-07-10,338,...,2017-08-10,1,007,LOW,NON FEE,CASHBACK,1721,1,0,2017-07-12


In [142]:
len(df_OPEN2)/130

74239.0

In [143]:
len(df_OPEN2[df_OPEN2['Account_Number']=="MAFF_ACCT_0000266356"])
len(df_OPEN2)/130

130

74239.0

In [144]:
del df_OPEN2['random_end_dt']

In [145]:
output = open('Open_Sampling_Ready1.pkl', 'wb')
pickle.dump(df_OPEN2, output)
output.close()

In [146]:
df.shape
df_OPEN2.shape
df_VOLUNTARY_Account_Number.shape
df_VOLUNTARY_Account_Number.head(2)

(29626685, 24)

(9651070, 24)

(6906, 1)

,Account_Number
11315,MAFF_ACCT_0000005355
16425,MAFF_ACCT_0000005375


In [147]:
df_VOLUNTARY = df[df['Account_Number'].isin(list(df_VOLUNTARY_Account_Number['Account_Number']))]

In [148]:
df_VOLUNTARY.shape
df_VOLUNTARY.Account_Number.nunique()
df_VOLUNTARY_Account_Number.values
len(df_VOLUNTARY)/365

(2507550, 24)

6870

array([['MAFF_ACCT_0000005355'],
       ['MAFF_ACCT_0000005375'],
       ['MAFF_ACCT_0000005394'],
       ...,
       ['MAFF_ACCT_0000266087'],
       ['MAFF_ACCT_0000266126'],
       ['MAFF_ACCT_0000266249']], dtype=object)

6870.0

In [149]:
df_VOLUNTARY = df_VOLUNTARY.reset_index(drop=True)
df_VOLUNTARY = df_VOLUNTARY.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False])
df_VOLUNTARY = df_VOLUNTARY.reset_index(drop=True)
df_VOLUNTARY = df_VOLUNTARY.groupby('Account_Number').head(130)
df_VOLUNTARY = df_VOLUNTARY.reset_index(drop=True)

In [150]:
len(df_VOLUNTARY)/130
df_VOLUNTARY.isnull().sum()

6870.0

Account_Number                        0
MISDATE                               0
ATTRITION_FLAG                        0
ACCOUNT_CREATION_DATE                 0
ACTUAL_CURRENT_BALANCE                0
Credit_Limit                          0
LAST_ACTIVITY_DATE                40467
Last_Statement_Closing_Balance        0
Last_Statement_Date                   0
Last_Statement_Minimum_Due            0
Monthly_Salary                        0
Months_On_Book                        0
Next_Statement_Date                   0
Monthly_Salary                        0
Nationality_Group                     0
Next_Statement_Date                   0
No_Of_Due_Stmt                        0
Product                               0
APR_Rate                              0
Product_Fee_Type                      0
Product_Category                      0
Days_On_Book                          0
Days_From_Statement                   0
Days_from_Last_Activity               0
dtype: int64

In [151]:
totalframe = pd.concat([df_VOLUNTARY,df_OPEN2], axis=0)
totalframe.shape
totalframe = pd.DataFrame(totalframe)

(10544170, 24)

In [152]:
totalframe.isnull().sum()

Account_Number                         0
MISDATE                                0
ATTRITION_FLAG                         0
ACCOUNT_CREATION_DATE                  0
ACTUAL_CURRENT_BALANCE                 0
Credit_Limit                           0
LAST_ACTIVITY_DATE                298217
Last_Statement_Closing_Balance         0
Last_Statement_Date                    0
Last_Statement_Minimum_Due             0
Monthly_Salary                         0
Months_On_Book                         0
Next_Statement_Date                    0
Monthly_Salary                         0
Nationality_Group                      0
Next_Statement_Date                    0
No_Of_Due_Stmt                         0
Product                                0
APR_Rate                               0
Product_Fee_Type                       0
Product_Category                       0
Days_On_Book                           0
Days_From_Statement                    0
Days_from_Last_Activity                0
dtype: int64

In [153]:
#Check to see our data is divisable by 130 timesteps
len(totalframe)/130

81109.0

In [154]:
#Write out sample
output = open('All_Sampling_Ready1.pkl', 'wb')
pickle.dump(totalframe, output)
output.close()

In [155]:
del df_VOLUNTARY
del df_OPEN2
totalframe.shape
totalframe.head(1)

(10544170, 24)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category,Days_On_Book,Days_From_Statement,Days_from_Last_Activity
0,MAFF_ACCT_0000005355,2017-04-09,VOLUNTARY,2013-04-16,-0.3,25313,2017-04-08,262,2017-04-05,100,...,01. INDIA,2017-05-05,0,007,LOW,NON FEE,CASHBACK,1454,4,1


In [156]:
Statements_cleaned = pd.read_csv('Statements_cleaned.csv',sep=',',encoding='utf-8')
del Statements_cleaned['Unnamed: 0']
Statements_cleaned.shape
Statements_cleaned.head(2)
Statements_cleaned.dtypes
#list(Statements_cleaned.columns.values)

CRM_cleaned = pd.read_csv('CRM_cleaned.csv',sep=',',encoding='utf-8')
del CRM_cleaned['Unnamed: 0']
CRM_cleaned.shape
CRM_cleaned.head(2)
CRM_cleaned.dtypes
#list(CRM_cleaned.columns.values)

(2129999, 8)

,Account_Number,STATEMENT_DATE,STATEMENT_FROM_DATE,STATEMENT_TO_DATE,STATEMENT_BALANCE,POINTS_REDEEMED,POINTS_BALANCE_CLOSING,POINTS_EARNED
0,MAFF_ACCT_0000000001,2017-05-03,2017-04-03,2017-05-03,0,0,0,0
1,MAFF_ACCT_0000000003,2017-12-18,2017-11-18,2017-12-18,2049,0,0,0


Account_Number            object
STATEMENT_DATE            object
STATEMENT_FROM_DATE       object
STATEMENT_TO_DATE         object
STATEMENT_BALANCE          int64
POINTS_REDEEMED            int64
POINTS_BALANCE_CLOSING     int64
POINTS_EARNED              int64
dtype: object

(2102570, 4)

,Account_Number,Incident_Category,Incident_Type,Created_On
0,MAFF_ACCT_0000000002,Voyager Booking,Voyager,2017-08-02
1,MAFF_ACCT_0000000002,Cancellations,Life Style Protection (LSP),2017-08-02


Account_Number       object
Incident_Category    object
Incident_Type        object
Created_On           object
dtype: object

In [157]:
Transaction_DailyAgg_cleaned = pd.read_csv('Transactions_DailyAgg_cleaned.csv',sep=',',encoding='utf-8')
del Transaction_DailyAgg_cleaned['Unnamed: 0']
Transaction_DailyAgg_cleaned.shape
Transaction_DailyAgg_cleaned.head(2)
Transaction_DailyAgg_cleaned.dtypes

(15097872, 4)

,Account_Number,Purchase_Date,Billing_Amount_Sum,Transaction_Count
0,MAFF_ACCT_0000032090,2016-02-14,102.0,1.0
1,MAFF_ACCT_0000073466,2017-11-21,438.0,1.0


Account_Number         object
Purchase_Date          object
Billing_Amount_Sum    float64
Transaction_Count     float64
dtype: object

In [158]:
Transaction_DailyAgg_cleaned = Transaction_DailyAgg_cleaned.dropna(how='any',axis=0)

Statements_cleaned.isnull().sum()
CRM_cleaned.isnull().sum()
Transaction_DailyAgg_cleaned.isnull().sum()

Account_Number            0
STATEMENT_DATE            0
STATEMENT_FROM_DATE       0
STATEMENT_TO_DATE         0
STATEMENT_BALANCE         0
POINTS_REDEEMED           0
POINTS_BALANCE_CLOSING    0
POINTS_EARNED             0
dtype: int64

Account_Number       0
Incident_Category    0
Incident_Type        0
Created_On           0
dtype: int64

Account_Number        0
Purchase_Date         0
Billing_Amount_Sum    0
Transaction_Count     0
dtype: int64

#### Format Select Variables

In [159]:
CRM_cleaned['Account_Number'] = CRM_cleaned['Account_Number'].astype(str)
CRM_cleaned['Created_On'] = CRM_cleaned['Created_On'].astype('datetime64[ns]')

In [160]:
del Portfolio_cleaned

#### Join Key Incident Data from CRM to Portfolio Dataframe.

In [161]:
CRM_cleaned.shape
threshold = 200
counts_type = CRM_cleaned['Incident_Type'].value_counts()
counts_cat = CRM_cleaned['Incident_Category'].value_counts()

CRM_cleaned = CRM_cleaned.loc[CRM_cleaned['Incident_Type'].isin(counts_type[counts_type > threshold].index), :]
CRM_cleaned = CRM_cleaned.loc[CRM_cleaned['Incident_Category'].isin(counts_cat[counts_cat > threshold].index), :]
CRM_cleaned = CRM_cleaned.reset_index(drop=True)
CRM_cleaned.shape


(2102570, 4)

(2095876, 4)

#### Truncate CRM Daily Incident Data to 5 Most Recent Per Day

In [162]:
# Truncate to 5 incidents per account per date
CRM_cleaned = CRM_cleaned.sort_values(['Account_Number','Created_On'], ascending=[True, True])
CRM_cleaned = CRM_cleaned.reset_index(drop=True)

Account_Incident_Counts = CRM_cleaned.groupby(['Account_Number', 'Created_On']).size()
Account_Incident_Counts = Account_Incident_Counts.reset_index()
Account_Incident_Counts.columns = ['Account_Number', 'Created_On', 'Incident_Num']
Account_Incident_Counts = pd.DataFrame(Account_Incident_Counts)
Account_Incident_Counts.head(1)

CRM_cleaned.shape
CRM_cleaned2 = CRM_cleaned.sort_values(['Account_Number','Created_On'], ascending=[True, True])
CRM_cleaned2 = CRM_cleaned2.groupby(['Account_Number','Created_On']).head(5)                                                         
CRM_cleaned2 = CRM_cleaned2.reset_index(drop=True)


,Account_Number,Created_On,Incident_Num
0,MAFF_ACCT_0000000002,2017-08-02,2


(2095876, 4)

In [163]:
Account_Incident_Counts.isnull().sum()

Account_Number    0
Created_On        0
Incident_Num      0
dtype: int64

#### Concatanate sequences of the category and type of the day's incidents for that customer

In [164]:
# Aggregate longer sequences of multiple incidents per day into its own variable.
CRM_cleaned_seq = CRM_cleaned.groupby(['Account_Number', 'Created_On'])['Incident_Category'].agg(lambda col: ' _ '.join(col))
CRM_cleaned_seq = pd.DataFrame(CRM_cleaned_seq)
CRM_cleaned_seq = CRM_cleaned_seq.reset_index(drop=True)
CRM_cleaned_seq.columns = ['Incident_Cat_Sequence_Daily']
CRM_cleaned_seq.shape

# Aggregate longer sequences of multiple incidents per day into its own variable.
CRM_cleaned_seq2 = CRM_cleaned.groupby(['Account_Number', 'Created_On'])['Incident_Type'].agg(lambda col: ' _ '.join(col))
CRM_cleaned_seq2 = CRM_cleaned_seq2.reset_index(drop=True)
CRM_cleaned_seq2 = pd.DataFrame(CRM_cleaned_seq2)
CRM_cleaned_seq2.columns = ['Incident_Type_Sequence_Daily']
CRM_cleaned_seq2.shape

(1396983, 1)

(1396983, 1)

#### Join the new variables to the data frame.

In [165]:
CRM_cleaned_seq = CRM_cleaned_seq.join(CRM_cleaned_seq2)
CRM_cleaned_seq.head(4)

CRM_cleaned = CRM_cleaned.sort_values(['Account_Number','Created_On'], ascending=[True, True])
CRM_Index = CRM_cleaned.groupby(['Account_Number', 'Created_On']).head(1)
CRM_Index = pd.DataFrame(CRM_Index)
CRM_Index = CRM_Index.reset_index(drop=True)
CRM_Index.shape

CRM_Joined = CRM_Index.join(CRM_cleaned_seq)
CRM_Joined.head(4)

,Incident_Cat_Sequence_Daily,Incident_Type_Sequence_Daily
0,Voyager Booking _ Cancellations,Voyager _ Life Style Protection (LSP)
1,Annual Membership Fee,Reversal- New
2,Real Time Update,Address / Contact details update
3,Cancellations _ Annual Membership Fee,Life Style Protection (LSP) _ Reversal- New


(1396983, 4)

,Account_Number,Incident_Category,Incident_Type,Created_On,Incident_Cat_Sequence_Daily,Incident_Type_Sequence_Daily
0,MAFF_ACCT_0000000002,Voyager Booking,Voyager,2017-08-02,Voyager Booking _ Cancellations,Voyager _ Life Style Protection (LSP)
1,MAFF_ACCT_0000000002,Annual Membership Fee,Reversal- New,2017-08-30,Annual Membership Fee,Reversal- New
2,MAFF_ACCT_0000000003,Real Time Update,Address / Contact details update,2017-07-27,Real Time Update,Address / Contact details update
3,MAFF_ACCT_0000000003,Cancellations,Life Style Protection (LSP),2017-08-02,Cancellations _ Annual Membership Fee,Life Style Protection (LSP) _ Reversal- New


#### Merge CRM Incident Daily Incidents and Sequences to the Working File

In [166]:
Merged2 = pd.merge(totalframe, CRM_Joined,  how='left', 
                  left_on=['Account_Number','MISDATE'], 
                  right_on = ['Account_Number','Created_On'])
del Merged2['Created_On']

In [167]:
del totalframe

In [168]:
Merged2['Incident_Category'].fillna('None', inplace=True)
Merged2['Incident_Type'].fillna('None', inplace=True)
Merged2['Incident_Cat_Sequence_Daily'].fillna('None', inplace=True)
Merged2['Incident_Type_Sequence_Daily'].fillna('None', inplace=True)
Merged2['Incident_Category']=Merged2['Incident_Category'].replace(-np.nan, 'None').replace(np.nan, 'None')
Merged2['Incident_Type']=Merged2['Incident_Type'].replace(-np.nan, 'None').replace(np.nan, 'None')
Merged2['Incident_Cat_Sequence_Daily']=Merged2['Incident_Cat_Sequence_Daily'].replace(-np.nan, 'None').replace(np.nan, 'None')
Merged2['Incident_Type_Sequence_Daily']=Merged2['Incident_Type_Sequence_Daily'].replace(-np.nan, 'None').replace(np.nan,'None')

In [169]:
Merged2['Incident_Category'] = Merged2['Incident_Category'].astype('str')
Merged2['Incident_Type'] = Merged2['Incident_Type'].astype('str')
Merged2['Incident_Cat_Sequence_Daily'] = Merged2['Incident_Cat_Sequence_Daily'].astype('str')
Merged2['Incident_Type_Sequence_Daily'] = Merged2['Incident_Type_Sequence_Daily'].astype('str')

In [170]:
Merged2 = pd.merge(Merged2, Account_Incident_Counts,  how='left', 
                  left_on=['Account_Number','MISDATE'], 
                  right_on = ['Account_Number','Created_On'])
del Merged2['Created_On']

In [171]:
Merged2['Incident_Num'].fillna(0, inplace=True)            
Merged2['Incident_Num'] =Merged2['Incident_Num'].replace(np.nan,0).replace(np.inf,0) 

In [172]:
Merged2.isnull().sum()

Account_Number                         0
MISDATE                                0
ATTRITION_FLAG                         0
ACCOUNT_CREATION_DATE                  0
ACTUAL_CURRENT_BALANCE                 0
Credit_Limit                           0
LAST_ACTIVITY_DATE                298217
Last_Statement_Closing_Balance         0
Last_Statement_Date                    0
Last_Statement_Minimum_Due             0
Monthly_Salary                         0
Months_On_Book                         0
Next_Statement_Date                    0
Monthly_Salary                         0
Nationality_Group                      0
Next_Statement_Date                    0
No_Of_Due_Stmt                         0
Product                                0
APR_Rate                               0
Product_Fee_Type                       0
Product_Category                       0
Days_On_Book                           0
Days_From_Statement                    0
Days_from_Last_Activity                0
Incident_Categor

In [173]:
Merged2.columns
Merged2.head(2)

Index(['Account_Number', 'MISDATE', 'ATTRITION_FLAG', 'ACCOUNT_CREATION_DATE',
       'ACTUAL_CURRENT_BALANCE', 'Credit_Limit', 'LAST_ACTIVITY_DATE',
       'Last_Statement_Closing_Balance', 'Last_Statement_Date',
       'Last_Statement_Minimum_Due', 'Monthly_Salary', 'Months_On_Book',
       'Next_Statement_Date', 'Monthly_Salary', 'Nationality_Group',
       'Next_Statement_Date', 'No_Of_Due_Stmt', 'Product', 'APR_Rate',
       'Product_Fee_Type', 'Product_Category', 'Days_On_Book',
       'Days_From_Statement', 'Days_from_Last_Activity', 'Incident_Category',
       'Incident_Type', 'Incident_Cat_Sequence_Daily',
       'Incident_Type_Sequence_Daily', 'Incident_Num'],
      dtype='object')

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Product_Fee_Type,Product_Category,Days_On_Book,Days_From_Statement,Days_from_Last_Activity,Incident_Category,Incident_Type,Incident_Cat_Sequence_Daily,Incident_Type_Sequence_Daily,Incident_Num
0,MAFF_ACCT_0000005355,2017-04-09,VOLUNTARY,2013-04-16,-0.3,25313,2017-04-08,262,2017-04-05,100,...,NON FEE,CASHBACK,1454,4,1,None,None,None,None,0.0
1,MAFF_ACCT_0000005355,2017-04-08,OPEN,2013-04-16,-0.3,25313,2017-04-08,262,2017-04-05,100,...,NON FEE,CASHBACK,1453,3,0,Charges,Card Cancellation,Charges,Card Cancellation,1.0


In [174]:
Merged2.shape

(10544170, 29)

In [175]:
output = open('Intermediate_preprocessing2.pkl', 'wb')
pickle.dump(Merged2, output)
output.close()


### Create Select One-Hot Encodings for Select Variables

In [176]:
#APR Rate 
Merged2['APR_Rate'].unique()
Merged2['APR_Rate_Low'] = np.where(Merged2['APR_Rate']=='LOW', '1', '0')
Merged2['APR_Rate_Med'] = np.where(Merged2['APR_Rate']=='MEDIUM', '1', '0')
Merged2['APR_Rate_High'] = np.where(Merged2['APR_Rate']=='HIGH', '1', '0')
del Merged2['APR_Rate']

Merged2['APR_Rate_Low'] = Merged2['APR_Rate_Low'].replace(np.nan, 0).replace(np.inf, 0).astype(float)
Merged2['APR_Rate_Med'] = Merged2['APR_Rate_Med'].replace(np.nan, 0).replace(np.inf, 0).astype(float)
Merged2['APR_Rate_High'] = Merged2['APR_Rate_High'].replace(np.nan, 0).replace(np.inf, 0).astype(float)

Merged2['APR_Rate_Low'] = Merged2['APR_Rate_Low'].astype(int)
Merged2['APR_Rate_Med'] = Merged2['APR_Rate_Med'].astype(int)
Merged2['APR_Rate_High'] = Merged2['APR_Rate_High'].astype(int)


array(['LOW', 'MEDIUM', 'HIGH'], dtype=object)

In [177]:
#Product Category 
Merged2['Product_Category'].unique()
Merged2['Prod_Cat_Voyager'] = np.where(Merged2['Product_Category']=='VOYAGER', '1', '0')
Merged2['Prod_Cat_Cashback'] = np.where(Merged2['Product_Category']=='CASHBACK', '1', '0')
del Merged2['Product_Category']

Merged2['Prod_Cat_Voyager'] = Merged2['Prod_Cat_Voyager'].replace(np.nan, 0).replace(np.inf, 0).astype(float)
Merged2['Prod_Cat_Cashback'] = Merged2['Prod_Cat_Cashback'].replace(np.nan, 0).replace(np.inf, 0).astype(float)

Merged2['Prod_Cat_Voyager'] = Merged2['Prod_Cat_Voyager'].astype(int)
Merged2['Prod_Cat_Cashback'] = Merged2['Prod_Cat_Cashback'].astype(int)

array(['CASHBACK', 'VOYAGER'], dtype=object)

In [178]:
Merged2_backup = Merged2
Merged2['Nationality_Group'] = Merged2['Nationality_Group'].replace('01. INDIA', 'INDIA')
Merged2['Nationality_Group'] = Merged2['Nationality_Group'].replace('02. PAKISTAN', 'PAKISTAN')
Merged2['Nationality_Group'] = Merged2['Nationality_Group'].replace('03. PHILLIPPINES', 'PHILLIPPINES')
Merged2['Nationality_Group'] = Merged2['Nationality_Group'].replace('04. U.A.E.', 'U.A.E.')
Merged2['Nationality_Group'] = Merged2['Nationality_Group'].replace('05. ARAB', 'ARAB')
Merged2['Nationality_Group'] = Merged2['Nationality_Group'].replace('06. OTHERS', 'OTHERS')

In [179]:
del Merged2_backup

In [180]:
Merged2= pd.get_dummies(Merged2, columns=['Nationality_Group'], prefix=['Country'])


In [181]:
#Format new int variables as int
Merged2['Country_INDIA'] = Merged2['Country_INDIA'].replace(np.nan, 0).astype(int)
Merged2['Country_PAKISTAN'] = Merged2['Country_PAKISTAN'].replace(np.nan, 0).astype(int)
Merged2['Country_PHILLIPPINES'] = Merged2['Country_PHILLIPPINES'].replace(np.nan, 0).astype(int)
Merged2['Country_U.A.E.'] = Merged2['Country_U.A.E.'].replace(np.nan, 0).astype(int)
Merged2['Country_ARAB'] = Merged2['Country_ARAB'].replace(np.nan, 0).astype(int)
Merged2['Country_OTHERS'] = Merged2['Country_OTHERS'].replace(np.nan, 0).astype(int)



### Derived Features

In [183]:
Merged2['ACTUAL_CURRENT_BALANCE'] = Merged2['ACTUAL_CURRENT_BALANCE'].replace(np.inf, 1).replace(np.nan, 0).astype(float)
Merged2['Credit_Limit'] = Merged2['Credit_Limit'].replace(np.inf, 1).replace(np.nan, 0).astype(float)

Merged2['Credit_Limit'] = Merged2['Credit_Limit'].round().astype(int)
Merged2['ACTUAL_CURRENT_BALANCE'] = Merged2['ACTUAL_CURRENT_BALANCE'].round().astype(int)

In [184]:
# Percent Credit Used
Merged2['Perc_Credit_Limit_Used'] = (Merged2['ACTUAL_CURRENT_BALANCE'])/Merged2['Credit_Limit']
Merged2['Perc_Credit_Limit_Used'] = Merged2['Perc_Credit_Limit_Used'].round(2)
Merged2['Perc_Credit_Limit_Used'] = Merged2['Perc_Credit_Limit_Used'].replace(np.inf, 1).replace(np.nan, 0).astype(float)


In [185]:
del Merged2['ACCOUNT_CREATION_DATE']
del Merged2['Last_Statement_Date']
del Merged2['LAST_ACTIVITY_DATE']
del Merged2['Next_Statement_Date']

### Write out intermediate file

In [186]:
#Merged2.to_csv('Merged_cleaned.csv',sep=',',encoding='utf-8')
output = open('Merged_cleaned2.pkl', 'wb')
pickle.dump(Merged2, output)
output.close()

In [187]:
#Merged2.to_csv('Merged_cleaned.csv',sep=',',encoding='utf-8')
df1 = Merged2

In [188]:

del Merged2

### Import and Merge Statements Table Data of Interest

In [189]:
df2 = pd.read_csv('Statements_cleaned.csv',sep=',',encoding='utf-8')
del df2['Unnamed: 0']
del df2['STATEMENT_FROM_DATE']
del df2['STATEMENT_TO_DATE']
df2.head(2)

,Account_Number,STATEMENT_DATE,STATEMENT_BALANCE,POINTS_REDEEMED,POINTS_BALANCE_CLOSING,POINTS_EARNED
0,MAFF_ACCT_0000000001,2017-05-03,0,0,0,0
1,MAFF_ACCT_0000000003,2017-12-18,2049,0,0,0


In [190]:
df2['STATEMENT_DATE'] = df2['STATEMENT_DATE'].astype('datetime64[ns]')
df3 = pd.merge(df1,df2,how='left', 
                  left_on=['Account_Number','MISDATE'], 
                  right_on = ['Account_Number','STATEMENT_DATE'])
del df3['STATEMENT_DATE'] 
del df3['STATEMENT_BALANCE']
df3['POINTS_REDEEMED']=df3['POINTS_REDEEMED'].replace(np.nan, 0).replace(np.nan, 0)
df3['POINTS_BALANCE_CLOSING']=df3['POINTS_BALANCE_CLOSING'].replace(-np.nan, 0).replace(np.nan, 0)
df3['POINTS_EARNED']=df3['POINTS_EARNED'].replace(-np.nan, 0).replace(np.nan, 0)

df3['POINTS_REDEEMED'].fillna(0, inplace=True)
df3['POINTS_BALANCE_CLOSING'].fillna(0, inplace=True)
df3['POINTS_EARNED'].fillna(0, inplace=True)

df3['POINTS_REDEEMED'] = df3['POINTS_REDEEMED'].astype('int')
df3['POINTS_BALANCE_CLOSING']= df3['POINTS_BALANCE_CLOSING'].astype('int')
df3['POINTS_EARNED']= df3['POINTS_EARNED'].astype('int')
df3.shape

(10544170, 36)

In [191]:
del df2

In [192]:
os.chdir('D://MAFData//')
os.getcwd()

with open('Transactions_cleaned.pkl', 'rb') as f:
    Transactions_DailyAgg_cleaned = pickle.load(f)


'D:\\MAFData'

In [194]:
df1['Account_Number'] = df1['Account_Number'].astype('str')
df1['Account_Number'] = df1['Account_Number'].astype('str')

Transactions_DailyAgg_cleaned['Purchase_Date'] = Transactions_DailyAgg_cleaned['Purchase_Date'].astype('datetime64[ns]')
df1['MISDATE'] = df1['MISDATE'].astype('datetime64[ns]')

Transactions_DailyAgg_cleaned = Transactions_DailyAgg_cleaned[pd.notnull(Transactions_DailyAgg_cleaned['Account_Number'])] 
Transactions_DailyAgg_cleaned = Transactions_DailyAgg_cleaned[pd.notnull(Transactions_DailyAgg_cleaned['Purchase_Date'])] 

df1.tail(1)
Transactions_DailyAgg_cleaned.tail(1)

,Account_Number,MISDATE,ATTRITION_FLAG,ACTUAL_CURRENT_BALANCE,Credit_Limit,Last_Statement_Closing_Balance,Last_Statement_Minimum_Due,Monthly_Salary,Months_On_Book,Monthly_Salary,...,APR_Rate_High,Prod_Cat_Voyager,Prod_Cat_Cashback,Country_ARAB,Country_INDIA,Country_OTHERS,Country_PAKISTAN,Country_PHILLIPPINES,Country_U.A.E.,Perc_Credit_Limit_Used
10544169,MAFF_ACCT_0000266356,2016-12-25,OPEN,11763,30000,11763,588,700000,3,700000,...,0,1,0,0,1,0,0,0,0,0.39


,Account_Number,Purchase_Date,Billing_Amount_Sum_Payment,Billing_Amount_Sum_Spend_ESS,Billing_Amount_Sum_Spend_NON_ESS,Billing_Amount_Spend_CASH,Billing_Amount_Violation_Fee,Transaction_Count_Payment,Tranasaction_Count_Spend_ESS,Tranasaction_Count_Spend_NON_ESS,Tranasaction_Count_Spend_CASH,Transaction_Count_Foreclosure,Tranasaction_Count_BT,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee
15097870,MAFF_ACCT_0000036399,2017-04-30,-200.000,0.000,0.000,0.000,0.000,1,0,0,0,0,0,0,0,0


In [195]:

df3 = pd.merge(df3,Transactions_DailyAgg_cleaned,how='left', 
                  left_on=['Account_Number','MISDATE'], 
                  right_on = ['Account_Number','Purchase_Date'])
del df3['Purchase_Date']

In [196]:
del df1

In [197]:
df4 = df3
del df3

In [198]:
#Merged2.to_csv('Merged_cleaned.csv',sep=',',encoding='utf-8')
output = open('Merged_cleaned2.pkl', 'wb')
pickle.dump(df4, output)
output.close()

### Replace remaining nan and infinities with '0'

In [200]:
df4['Billing_Amount_Sum_Spend_ESS']=df4['Billing_Amount_Sum_Spend_ESS'].replace(-np.nan, 0).replace(np.nan, 0)
df4['Billing_Amount_Sum_Spend_NON_ESS']=df4['Billing_Amount_Sum_Spend_NON_ESS'].replace(-np.nan, 0).replace(np.nan, 0)
df4['Billing_Amount_Spend_CASH']=df4['Billing_Amount_Spend_CASH'].replace(np.nan, 0).replace(np.nan, 0)
df4['Billing_Amount_Violation_Fee']=df4['Billing_Amount_Violation_Fee'].replace(-np.nan, 0).replace(np.nan, 0)
df4['Transaction_Count_Payment']=df4['Transaction_Count_Payment'].replace(-np.nan, 0).replace(np.nan, 0)

In [201]:
df4['Tranasaction_Count_Spend_ESS']=df4['Tranasaction_Count_Spend_ESS'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Tranasaction_Count_Spend_NON_ESS']=df4['Tranasaction_Count_Spend_NON_ESS'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Tranasaction_Count_Spend_CASH']=df4['Tranasaction_Count_Spend_CASH'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Transaction_Count_Foreclosure']=df4['Transaction_Count_Foreclosure'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Tranasaction_Count_BT']=df4['Tranasaction_Count_BT'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Tranasaction_Count_LOP']=df4['Tranasaction_Count_LOP'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Tranasaction_Count_EPP']=df4['Tranasaction_Count_EPP'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Tranasaction_Count_Violation_Fee']=df4['Tranasaction_Count_Violation_Fee'].replace(np.nan, 0).replace(np.inf, 0).astype(int)

In [203]:
df4.head(1)

,Account_Number,MISDATE,ATTRITION_FLAG,ACTUAL_CURRENT_BALANCE,Credit_Limit,Last_Statement_Closing_Balance,Last_Statement_Minimum_Due,Monthly_Salary,Months_On_Book,Monthly_Salary,...,Billing_Amount_Violation_Fee,Transaction_Count_Payment,Tranasaction_Count_Spend_ESS,Tranasaction_Count_Spend_NON_ESS,Tranasaction_Count_Spend_CASH,Transaction_Count_Foreclosure,Tranasaction_Count_BT,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee
0,MAFF_ACCT_0000005355,2017-04-09,VOLUNTARY,0,25313,262,100,20708,48,20708,...,0,0,0,0,0,0,0,0,0,0


In [204]:
df4['Incident_Num']=df4['Incident_Num'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['APR_Rate_Low']=df4['APR_Rate_Low'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['APR_Rate_Med']=df4['APR_Rate_Med'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['APR_Rate_High']=df4['APR_Rate_High'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Prod_Cat_Voyager']=df4['Prod_Cat_Voyager'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Prod_Cat_Cashback']=df4['Prod_Cat_Cashback'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Days_On_Book']=df4['Days_On_Book'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Days_From_Statement']=df4['Days_From_Statement'].replace(np.nan, 0).replace(np.inf, 0).astype(int)
df4['Days_from_Last_Activity']=df4['Days_from_Last_Activity'].replace(np.nan, 0).replace(np.inf, 0).astype(int)

In [205]:
df4.dtypes

Account_Number                              object
MISDATE                             datetime64[ns]
ATTRITION_FLAG                              object
ACTUAL_CURRENT_BALANCE                       int32
Credit_Limit                                 int32
Last_Statement_Closing_Balance               int32
Last_Statement_Minimum_Due                   int32
Monthly_Salary                               int32
Months_On_Book                               int32
Monthly_Salary                               int32
No_Of_Due_Stmt                              object
Product                                     object
Product_Fee_Type                            object
Days_On_Book                                 int32
Days_From_Statement                          int32
Days_from_Last_Activity                      int32
Incident_Category                           object
Incident_Type                               object
Incident_Cat_Sequence_Daily                 object
Incident_Type_Sequence_Daily   

### Add Timestep Variable

In [206]:
df4 = df4.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False])
df4 = df4.reset_index(drop=True)
df4['Timestep'] = df4.groupby(['Account_Number']).cumcount()+1
df4.shape
df4.head(4)
df4.tail(4)

(10544170, 51)

,Account_Number,MISDATE,ATTRITION_FLAG,ACTUAL_CURRENT_BALANCE,Credit_Limit,Last_Statement_Closing_Balance,Last_Statement_Minimum_Due,Monthly_Salary,Months_On_Book,Monthly_Salary,...,Transaction_Count_Payment,Tranasaction_Count_Spend_ESS,Tranasaction_Count_Spend_NON_ESS,Tranasaction_Count_Spend_CASH,Transaction_Count_Foreclosure,Tranasaction_Count_BT,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee,Timestep
0,MAFF_ACCT_0000002125,2017-07-12,OPEN,6911,18000,6775,338,9000,57,9000,...,0,2,2,0,0,0,0,0,0,1
1,MAFF_ACCT_0000002125,2017-07-11,OPEN,6822,18000,6775,338,9000,57,9000,...,0,1,1,0,0,0,0,0,0,2
2,MAFF_ACCT_0000002125,2017-07-10,OPEN,6775,18000,6775,338,9000,57,9000,...,0,0,0,0,0,0,0,0,0,3
3,MAFF_ACCT_0000002125,2017-07-09,OPEN,6775,18000,5139,3186,9000,57,9000,...,0,1,1,0,0,0,0,0,0,4


,Account_Number,MISDATE,ATTRITION_FLAG,ACTUAL_CURRENT_BALANCE,Credit_Limit,Last_Statement_Closing_Balance,Last_Statement_Minimum_Due,Monthly_Salary,Months_On_Book,Monthly_Salary,...,Transaction_Count_Payment,Tranasaction_Count_Spend_ESS,Tranasaction_Count_Spend_NON_ESS,Tranasaction_Count_Spend_CASH,Transaction_Count_Foreclosure,Tranasaction_Count_BT,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee,Timestep
10544166,MAFF_ACCT_0000266356,2016-12-28,OPEN,11763,30000,11763,588,700000,3,700000,...,0,0,0,0,0,0,0,0,0,127
10544167,MAFF_ACCT_0000266356,2016-12-27,OPEN,11763,30000,11763,588,700000,3,700000,...,0,0,0,0,0,0,0,0,0,128
10544168,MAFF_ACCT_0000266356,2016-12-26,OPEN,11763,30000,11763,588,700000,3,700000,...,0,0,0,0,0,0,0,0,0,129
10544169,MAFF_ACCT_0000266356,2016-12-25,OPEN,11763,30000,11763,588,700000,3,700000,...,0,0,0,0,0,0,0,0,0,130


### Save Local Copy

In [207]:

output = open('Merged_cleaned2.pkl', 'wb')
pickle.dump(df4, output)
output.close()

In [208]:
#df4.to_csv('Merged_cleaned2.csv',sep=',',encoding='utf-8')

In [209]:
output = open('Merged_cleaned2_baknov2.pkl', 'wb')
pickle.dump(df4, output)
output.close()